# Login HuggingFace

In [3]:
from google.colab import userdata
hf_token = userdata.get("HUGGINGFACE_TOKEN")
from huggingface_hub import login
login(token=hf_token)

# Load LLM

In [4]:
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [5]:
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

# Prompting Techniques

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

## Zero-shot

In [23]:
template = """Phân loại cảm xúc của đoạn văn sau thành một trong ba nhãn: Tích cực, Tiêu cực hoặc Trung lập.
Chỉ trả về một từ duy nhất: Tích cực, Tiêu cực hoặc Trung lập.

Đoạn văn: {text}

Trả lời:"""
prompt = ChatPromptTemplate.from_template(template)

texts = [
    "Tôi thực sự yêu thích bộ phim! Diễn xuất thật tuyệt vời.",
    "Thời tiết hôm nay khá điển hình cho thời điểm này trong năm.",
    "Tôi thất vọng với dịch vụ tôi nhận được tại nhà hàng."
]

def generate(prompt, text):
    formatted_prompt = prompt.format(text=text)  # Chuyển template thành chuỗi
    input_tokens = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)  # Tokenize
    output_tokens = model.generate(**input_tokens, max_new_tokens=3, eos_token_id=tokenizer.eos_token_id)  # Generate output
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)  # Giải mã output
    if "Trả lời:" in output_text:
        output_text = output_text.split("Trả lời:")[-1].strip()
    return output_text

for text in texts:
    result = generate(prompt, text)
    print(f"Văn bản: {text}")
    print(f"Cảm xúc: {result}")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Văn bản: Tôi thực sự yêu thích bộ phim! Diễn xuất thật tuyệt vời.
Cảm xúc: Tích cực


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Văn bản: Thời tiết hôm nay khá điển hình cho thời điểm này trong năm.
Cảm xúc: Trung lập.
Văn bản: Tôi thất vọng với dịch vụ tôi nhận được tại nhà hàng.
Cảm xúc: Tiêu cực


## Few-shots

In [ ]:
import torch
torch.cuda.empty_cache()

In [20]:
template = """
        Thực hiện nhiệm vụ được chỉ định theo từng văn bản sau.

        Ví dụ 1:
        Nhiệm vụ: phân tích cảm xúc
        Văn bản: Tôi rất thích sản phẩm này! Nó thật tuyệt vời.
        Trả lời: Tích cực

        Ví dụ 2:
        Nhiệm vụ: nhận diện ngôn ngữ
        Văn bản: Bonjour, comment allez-vous?
        Trả lời: Tiếng Pháp

        Bây giờ, thực hiện nhiệm vụ sau:
        Nhiệm vụ: {task}
        Văn bản: {text}
        Trả lời:
        """
prompt = ChatPromptTemplate.from_template(template)
input = [["Tôi không thể tin được điều này tuyệt vời đến mức nào!", "phân tích cảm xúc"],
         ["Guten Tag, wie geht es Ihnen?", "nhận diện ngôn ngữ"]]
def generate(text, task):
    formatted_prompt = prompt.format(text=text, task=task)  # Chuyển template thành chuỗi
    input_tokens = tokenizer(formatted_prompt, return_tensors="pt", truncation=True).to(model.device)  # Tokenize
    output_tokens = model.generate(**input_tokens, max_new_tokens=4)  # Generate output
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)  # Giải mã output
    if "Trả lời:" in output_text:
        output_text = output_text.split("Trả lời:")[-1].strip()
    return output_text

for text, task in input:
    result = generate(text, task)
    print(f"Văn bản: {text}")
    print(f"Kết quả: {result}\n")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Văn bản: Tôi không thể tin được điều này tuyệt vời đến mức nào!
Kết quả: - Tích cực

Văn bản: Guten Tag, wie geht es Ihnen?
Kết quả: - Tiếng Đức



## Instruction prompting

In [ ]:
def generate(instruction):
  input_tokens = tokenizer(instruction, return_tensors="pt").to(model.device)
  output_tokens = model.generate(**input_tokens, max_new_tokens=512)
  output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
  return output_text

In [ ]:
final_instruction = """
Nhiệm vụ: Tạo một kế hoạch bài giảng ngắn gọn để dạy tài chính cá nhân cơ bản cho học sinh trung học.

Hướng dẫn:
1. Bắt đầu với phần giới thiệu ngắn gọn giải thích tầm quan trọng của tài chính cá nhân.
2. Liệt kê 3-5 chủ đề chính cần đề cập (ví dụ: lập ngân sách, tiết kiệm, hiểu về tín dụng).
3. Với mỗi chủ đề:
   a) Cung cấp một lời giải thích ngắn gọn, phù hợp với lứa tuổi học sinh trung học.
   b) Đề xuất một hoạt động thực tế hoặc bài tập để củng cố khái niệm.
4. Kết luận với phần tóm tắt và đề xuất tài liệu học tập bổ sung.

Định dạng phản hồi theo dạng dàn ý có cấu trúc. Hướng tới sự rõ ràng và thu hút,
cân bằng giữa các ví dụ cụ thể và nguyên tắc chung có thể áp dụng cho nhiều tình huống tài chính khác nhau.
Giữ độ dài bài giảng khoảng 300 từ.
"""

print("Đầu ra của hướng dẫn cuối cùng:")
print(generate(final_instruction))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Đầu ra của hướng dẫn cuối cùng:

Nhiệm vụ: Tạo một kế hoạch bài giảng ngắn gọn để dạy tài chính cá nhân cơ bản cho học sinh trung học.

Hướng dẫn:
1. Bắt đầu với phần giới thiệu ngắn gọn giải thích tầm quan trọng của tài chính cá nhân.
2. Liệt kê 3-5 chủ đề chính cần đề cập (ví dụ: lập ngân sách, tiết kiệm, hiểu về tín dụng).
3. Với mỗi chủ đề:
   a) Cung cấp một lời giải thích ngắn gọn, phù hợp với lứa tuổi học sinh trung học.
   b) Đề xuất một hoạt động thực tế hoặc bài tập để củng cố khái niệm.
4. Kết luận với phần tóm tắt và đề xuất tài liệu học tập bổ sung.

Định dạng phản hồi theo dạng dàn ý có cấu trúc. Hướng tới sự rõ ràng và thu hút,
cân bằng giữa các ví dụ cụ thể và nguyên tắc chung có thể áp dụng cho nhiều tình huống tài chính khác nhau.
Giữ độ dài bài giảng khoảng 300 từ.
 

Dưới đây là dàn ý cho bài giảng về tài chính cá nhân cho học sinh trung học:

I. Giới thiệu (5 phút)

* Giải thích tầm quan trọng của tài chính cá nhân trong cuộc sống hàng ngày
* Khẳng định mục tiêu củ

## Chain of Thought (CoT)
  - Zero-shot, few-shots hoạt động tốt trên 1 số nhiệm vụ đơn giản, nhưng không tốt trên nhiều nhiệm vụ suy luận phức tạp

In [31]:
def generate(prompt, question):
    formatted_prompt = prompt.format(question=question)
    input_tokens = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    output_tokens = model.generate(**input_tokens, max_new_tokens=256)
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    if "Trả lời:" in output_text:
        output_text = output_text.split("Trả lời:")[-1].strip()
    return output_text

### Zero-shot CoT

In [25]:
template = """Giải bài toán sau từng bước. Đối với mỗi bước:
1. Nêu rõ bạn sẽ tính toán điều gì
2. Viết công thức bạn sẽ sử dụng (nếu có)
3. Thực hiện phép tính
4. Giải thích kết quả

Câu hỏi: {question}

Trả lời:"""
prompt = ChatPromptTemplate.from_template(template)
question = "Một chiếc xe ô tô di chuyển 150 km với tốc độ 60 km/h, sau đó thêm 100 km với tốc độ 50 km/h. Tốc độ trung bình cho toàn bộ hành trình là bao nhiêu?"
generate(prompt, question)
print(f"Trả lời: {result}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Trả lời: Bước 1: Tính thời gian di chuyển trong hai phần của hành trình

- Thời gian di chuyển 150 km với tốc độ 60 km/h: t1 = 150/60 = 2,5 giờ

- Thời gian di chuyển 100 km với tốc độ 50 km/h: t2 = 100/50 = 2 giờ

Bước 2: Tính tổng khoảng cách di chuyển

- Tổng khoảng cách di chuyển là 150 + 100 = 250 km

Bước 3: Tính tổng thời gian di chuyển

- Tổng thời gian di chuyển là t1 + t2 = 2,5 + 2 = 4,5 giờ

Bước 4: Tính tốc độ trung bình cho toàn bộ hành trình

- Tốc độ trung bình = Tổng khoảng cách di chuyển / Tổng thời gian di chuyển = 250 / 4,5 = 55,55 km/h

Vậy tốc độ trung bình cho toàn bộ hành trình là 55,55 km/h. 

Lưu ý: Để tính được tốc độ trung bình, ta cần phải tính được tổng khoảng cách và tổng thời gian di chuyển. Tốc độ trung bình là tổng khoảng cách di chuyển chia cho tổng thời


### Few-shots CoT

In [33]:
template = """
Hỏi: Tôi có 4 quả cam, mua thêm 3 quả, sau đó cho đi 2 quả. Tôi còn lại bao nhiêu quả?
Trả lời: Ban đầu có 4 quả cam. Mua thêm 3 quả, tổng là 7 quả. Sau đó, cho đi 2 quả, còn lại 5 quả.
###
Hỏi: Tôi có 6 quả cam, mua thêm 2 quả, sau đó cho đi 4 quả. Tôi còn lại bao nhiêu quả?
Trả lời: Ban đầu có 6 quả cam. Mua thêm 2 quả, tổng là 8 quả. Sau đó, cho đi 4 quả, còn lại 4 quả.
###
{question}
Trả lời:
"""
prompt = ChatPromptTemplate.from_template(template)
question = "Tôi có 3 quả cam, mua thêm 5 quả, sau đó cho đi 2 quả. Tôi còn lại bao nhiêu quả?"
result = generate(prompt, question)
print(f"Hỏi: {question}")
print(f"Trả lời: {result}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Hỏi: Tôi có 3 quả cam, mua thêm 5 quả, sau đó cho đi 2 quả. Tôi còn lại bao nhiêu quả?
Trả lời: + Ban đầu có 3 quả cam.
+ Mua thêm 5 quả, tổng là 8 quả.
+ Sau đó, cho đi 2 quả, còn lại 6 quả.


### Trigger

In [36]:
# Hãy suy nghĩ từng bước
template = """Giải bài toán sau 1 cách ngắn gọn, không lặp lại. Hãy suy nghĩ từng bước:
Câu hỏi: {question}

Trả lời:"""
prompt = ChatPromptTemplate.from_template(template)
question = "Một chiếc xe ô tô di chuyển 150 km với tốc độ 60 km/h, sau đó thêm 100 km với tốc độ 50 km/h. Tốc độ trung bình cho toàn bộ hành trình là bao nhiêu?"
result = generate(prompt, question)
print(f"Trả lời: {result}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Trả lời: Bước 1: Để tính tốc độ trung bình cho toàn bộ hành trình, ta cần tính tổng thời gian di chuyển và tổng khoảng cách di chuyển.

Bước 2: Thời gian di chuyển cho từng đoạn đường là:
- Đoạn 1: 150 km / 60 km/h = 2,5 giờ
- Đoạn 2: 100 km / 50 km/h = 2 giờ

Bước 3: Tổng thời gian di chuyển là thời gian của cả hai đoạn đường: 2,5 + 2 = 4,5 giờ.

Bước 4: Tổng khoảng cách di chuyển là tổng khoảng cách của cả hai đoạn đường: 150 + 100 = 250 km.

Bước 5: Tốc độ trung bình cho toàn bộ hành trình là tổng khoảng cách di chuyển chia cho tổng thời gian di chuyển: 250 km / 4,5 giờ = 55,56 km/h. 

Vậy, tốc độ trung bình cho toàn bộ hành trình là 55,56 km/h. 

Đáp án: 55,56.


## Tree-of-thought (ToT)

In [38]:
def generate(prompt, question):
    formatted_prompt = prompt.format(question=question)
    input_tokens = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
    output_tokens = model.generate(**input_tokens, max_new_tokens=512)
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    if "Trả lời:" in output_text:
        output_text = output_text.split("Trả lời:")[-1].strip()
    return output_text

In [39]:
template = """Hãy đưa ra các giải pháp cho câu hỏi dưới đây và đánh giá trước khi đưa ra 1 câu trả lời cuối cùng cho câu hỏi.
Câu hỏi: {question}

Trả lời:"""
prompt = ChatPromptTemplate.from_template(template)
question = "Một người đang muốn học lập trình nhưng không biết bắt đầu từ đâu. Họ nên học ngôn ngữ lập trình nào đầu tiên?"
result = generate(prompt, question)
print(f"Trả lời: {result}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Trả lời: Có nhiều ngôn ngữ lập trình khác nhau, và việc lựa chọn ngôn ngữ đầu tiên phụ thuộc vào nhiều yếu tố như mục tiêu, kinh nghiệm và sở thích cá nhân của người học. Dưới đây là một số giải pháp và đánh giá cho từng ngôn ngữ lập trình:

1. **Python**: Python là ngôn ngữ lập trình phổ biến và dễ học, thường được sử dụng trong nhiều lĩnh vực như phát triển web, khoa học máy tính, và dữ liệu. Nó có syntax đơn giản, dễ hiểu và có nhiều thư viện lớn.
   Đánh giá: Python là lựa chọn tuyệt vời cho người mới bắt đầu, vì nó có thể giúp người học nhanh chóng phát triển các kỹ năng lập trình cơ bản.

2. **JavaScript**: JavaScript là ngôn ngữ lập trình được sử dụng rộng rãi trong phát triển web, tạo ra các ứng dụng web động và các ứng dụng di động. Nó có syntax tương tự như HTML và CSS.
   Đánh giá: JavaScript là lựa chọn tốt nếu người học muốn phát triển các ứng dụng web hoặc muốn học về lập trình di động.

3. **Java**: Java là ngôn ngữ lập trình mạnh mẽ và phổ biến, thường được sử dụng tron